In [1]:
#所需要的库
#The required libraries
import re
from bs4 import BeautifulSoup
from concurrent import futures
import sys
import time
import pandas as pd
import requests
import json
from fake_useragent import UserAgent
import urllib3.contrib.pyopenssl
import urllib3
import logging


# Set the base parameters
urllib3.disable_warnings()
logging.captureWarnings(True)
urllib3.contrib.pyopenssl.inject_into_urllib3()
HEADERS = {}
NUM_THREADS = 5
PRINT = False
proxies = None
ua = UserAgent()

class JobSpider(object):

    def __init__(self):
        self.headers = {"User-Agent": UserAgent().random,'Connection':'close'}
        self.datas = list()
        self.countforall = 0
        self.dataset=pd.DataFrame(columns=["职位名称","薪资","所在地", "职位简介","职位详情",
                                     "经验要求", "学历要求", "招聘人数",
                                     "公司名称", "类型","规模",  "领域", "公司详情"])
    #获取最大可爬取页数
    #Gets the maximum number of pages to climb
    
    def getMaxPage(self, url):
        response = requests.get(url, headers = self.headers)
        if response.status_code == 200:
            source = response.text
            soup = BeautifulSoup(source, "html.parser")
            maxPage = soup.find("input", {"id" : "hidTotalPage"})["value"]
            return  int(maxPage)
        else:
            print("Fail status: {}".format(response.status_code))
            return None
        
    def getDetailPages(self,url):
        job=[]
        company=[]
        count=0
        print("当前正在爬取: {}".format(url))
        response = requests.get(url, headers = self.headers)
        soup = BeautifulSoup(response.text, "html.parser")
        link_company = soup.find_all("div", class_ = "el")
        link_job = soup.find_all("div", class_ = "el")
        for i in link_company:
            link_c1 = i.find_all("span", class_="t2")
            for i in link_c1:
                try:
                    link_c2=i.find("a")["href"]
                    company.append(link_c2)
                except Exception as e:
                    print("爬取公司链接时", e)
        for i in link_job:
            link_j1 = i.find_all("p", class_="t1")
            for i in link_j1:
                try:
                    link_j2 = i.find("a")["href"]
                    job.append(link_j2)
                except Exception as e:
                    print("爬取工作链接时", e)
        return job,company
        
    #连接和数据记录函数 
    #Connection and data recording functions
    def parsePage(self, url):
        maxPage = self.getMaxPage(url)
        for pageNum in range(180, maxPage+1):
            url = "https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,{}.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=".format(pageNum)
            print("当前正在爬取: 第{}页".format(pageNum))
            joburl,comurl=self.getDetailPages(url)
            with futures.ThreadPoolExecutor(max_workers =10) as t:
                obj_list = []
                for i,j in zip(joburl,comurl):
                    obj = t.submit(self.detail,i,j)
                    obj_list.append(obj)
                for future in futures.as_completed(obj_list):
                    data = future.result()
                    print(f"main: {data}")
        self.dataset.to_csv("Jobs.csv",encoding='utf_8_sig', index=False)


    #数据获取函数    
    #Data acquisition function
    def detail(self,url1,url2):
        try: 
            r_job = requests.get(url1, headers = self.headers,verify=False)
            r_job.encoding = "gbk"
            r_com = requests.get(url2, headers = self.headers,verify=False)
            r_com.encoding = "gbk"
            bs_obj_com = BeautifulSoup(r_com.text, "html.parser")
            bs_obj_job = BeautifulSoup(r_job.text, "html.parser")
            if bs_obj_job is not None:
                try:
                    div_job = bs_obj_job.find("div", class_="cn")
                    div_title_job = div_job.find("h1")["title"]
                    div_salary_job =div_job.find("strong").get_text()
                    div_info_job = "".join(div_job.find("p", class_="msg ltype")["title"].split())
                    title_strs_job = div_info_job.split("|")
                    if "发布" in title_strs_job[4]:
                        loc_1_job = title_strs_job[0].split("-")[0]
                        exp_job = title_strs_job[1]
                        edu_job = title_strs_job[2]
                        num_job = title_strs_job[3].replace("招","").replace("人","")
                        div_tag_job = div_job.find_all("span", class_="sp4")
                        tag_job =""
                        for i in div_tag_job:
                            tag_job = tag_job+"|"+i.get_text()
                        div2_job = bs_obj_job.find("div", class_="bmsg job_msg inbox")
                        div_detail_job = div2_job.find_all("p")
                        detail_job=""
                        for i in div_detail_job:
                            detail_job=detail_job+"|"+i.get_text()
                        div_com = bs_obj_com.find("div", class_="tHeader tHCop")
                        div_title_com = div_com.find("h1")["title"]
                        div_info_com = "".join(div_com.find("p", class_="ltype")["title"].split())
                        title_strs_com = div_info_com.split("|")
                        type_com = title_strs_com[0]
                        staff = title_strs_com[1]
                        field_com = title_strs_com[2]
                        div2_com = bs_obj_com.find("div", class_="con_txt")
                        detail_com = "".join(div2_com.get_text().split())
                        temp_df = pd.DataFrame(data=[[div_title_job,div_salary_job,loc_1_job,tag_job,detail_job,
                                              exp_job, edu_job, num_job,div_title_com, type_com, staff, field_com, 
                                              detail_com]], 
                                    columns=["职位名称","薪资","所在地", "职位简介","职位详情",
                                             "经验要求", "学历要求", "招聘人数",
                                             "公司名称", "类型","规模",  "领域", "公司详情"])
                        self.dataset = self.dataset.append(temp_df)
                        self.countforall +=1
                        self.dataset.to_csv("Jobs.csv",encoding='utf_8_sig', index=False)
                        print("已爬取信息：", self.countforall)
                except Exception as e:
                    print(e)
        except Exception as e:
            print(e)
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            time.sleep(5)
            print("Was a nice sleep, now let me continue...")
        return None
    
        
if __name__ == "__main__":
    Jobs = JobSpider()
    Jobs.parsePage("https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,1.html?lang=c&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&ord_field=0&dibiaoid=0&line=&welfare=")

当前正在爬取: 第180页
当前正在爬取: https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,180.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
爬取公司链接时 'NoneType' object is not subscriptable
已爬取信息： 10
main: None
已爬取信息：已爬取信息： 10
 10
已爬取信息：main: None 10

main: None
main: None
已爬取信息： 10
main: None
已爬取信息： 10
main: None
已爬取信息： 10
main: None
已爬取信息： 10
main: None已爬取信息： 10

main: None
已爬取信息： 10
main: None
已爬取信息： 20
已爬取信息： 20
main: None
main: None
已爬取信息： 20
已爬取信息：main: None
 20
main: None
已爬取信息： 20
已爬取信息： 20
main: None
main: None
已爬取信息： 20
已爬取信息：main: None
 20
main: None已爬取信息：
 20
main: None
已爬取信息： 20
main: None
list index out of range
main: None
已爬取信息： 29
已爬取信息：main: None 29

main: None
已爬取信息： 29
main: None
已爬取信息： 29
main: None
已爬取信息： 29
main: None
已爬取信息： 29
main: N

爬取公司链接时 'NoneType' object is not subscriptable
main: None
已爬取信息： 226
main: None
已爬取信息： 226
main: None
已爬取信息： 226
已爬取信息： main: None
226
main: None
已爬取信息： 已爬取信息： 226
226main: None

main: None
已爬取信息： 226
main: None
已爬取信息： 226
main: None已爬取信息： 226

main: None
list index out of range
main: None
已爬取信息： 235
main: None
已爬取信息： 235
main: None
已爬取信息： 235
main: None
已爬取信息： 已爬取信息：235 235

main: None
main: None
已爬取信息： 235
main: None
已爬取信息： 235
main: None已爬取信息：
 235
main: None
已爬取信息： 235
main: None
list index out of range
main: Nonelist index out of range

list index out of range
main: None
main: None已爬取信息： 242

main: None
已爬取信息： 242
main: None
已爬取信息： 242
已爬取信息： 242
main: None
main: None
已爬取信息： 242
main: None已爬取信息：
 242
main: None已爬取信息： 
242
main: None
list index out of range
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 251
已爬取信息： 251
main: None
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 251
main: None
已爬取信息： 261
main: No

已爬取信息： 366
main: None
list index out of range
main: None
已爬取信息： 374
main: None
已爬取信息： 已爬取信息： 374
main: None
374
已爬取信息：main: None 374

main: None
已爬取信息： 374
main: None
已爬取信息： 374已爬取信息：
main: None 374

已爬取信息： 374
main: None
main: None
当前正在爬取: 第189页
当前正在爬取: https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,189.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
main: None
已爬取信息： 381
main: None
已爬取信息： 383
main: None
已爬取信息： 383
main: None已爬取信息：
 383
已爬取信息： 383
main: None
main: None已爬取信息：
 383
已爬取信息：main: None 383

main: None
已爬取信息： 383
main: None已爬取信息： 383

main: None
已爬取信息： 384
main: None
list index out of rangelist index out of range

main: None
main: None
已爬取信息： 391
main: No

爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
list index out of rangemain: None

main: None
list index out of range
main: None
已爬取信息： 593
main: None
已爬取信息：已爬取信息： 593
 593
main: None
main: None
已爬取信息： 593
main: None
已爬取信息： 593
main: None
已爬取信息： 593
main: None
已爬取信息： 593
main: None
list index out of range
main: None
list index out of range
main: None
已爬取信息： 601已爬取信息： 601
main: None

已爬取信息：main: None 601

main: None
已爬取信息： 601
main: None
已爬取信息： 601已爬取信息： 601

main: None
main: None
已爬取信息： 已爬取信息： 601
main: None
601
main: None
list index out of range
main: None
list index out of range
main: None
已爬取信息： 609
main: None
已爬取信息： 609
main: None
已爬取信息： 609
main: None已爬取信息：已爬取信息： 609
 
609main: None

main: None
已爬取信息：已爬取信息： 609
 609
main: None
main: None
已爬取信息： 609
main: None
list index out of range
main: None
已爬取信息： 618
已爬取信息： 618
main: None
main: None
已爬取信息： 618
main: None
已爬取信息：已爬取信息： 618
main: None
 618
已爬取信息： 618
main: None
main: None
已爬取信息：已爬取信息： 618
 main: None618

ma

已爬取信息：已爬取信息： 812
main: None 
812
main: None
已爬取信息： 812
main: None
已爬取信息： 812
main: None
已爬取信息： 812
main: None
已爬取信息：已爬取信息： 812
 main: None
812
main: None
已爬取信息： 812
已爬取信息： 812已爬取信息： 
812
main: None
main: None
main: None
list index out of rangelist index out of range

main: None
list index out of rangemain: None

main: None
已爬取信息： 819
main: None
已爬取信息： 819
main: None
已爬取信息： 819
main: None
已爬取信息： 819
main: None
已爬取信息： 819
已爬取信息：main: None
 819
main: None
已爬取信息： 819
main: None
已爬取信息： 820
main: None
list index out of range
list index out of rangemain: None

main: None
已爬取信息：已爬取信息：  828
main: None828

main: None
已爬取信息： 828
main: None
已爬取信息：已爬取信息： 828
 main: None
828
main: None
已爬取信息： 已爬取信息： 828
828main: None

main: None
已爬取信息： 828
main: None
list index out of range
main: None
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
已爬取信息： 835
main: None
当前正在爬取: 第200页
当前正在爬取: https://search.51job.com/list/0

...

爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
main: None
已爬取信息： 3045
已爬取信息：main: None 3045

main: None
已爬取信息： 3045已爬取信息： 3045

main: None
main: None
已爬取信息： 已爬取信息：3045
 main: None3045

main: None
已爬取信息： 3045已爬取信息：
 3045
main: None
main: None
已爬取信息： 3045
main: None
已爬取信息： 3045
main: None
list index out of range
main: None
已爬取信息： 3055
main: None
已爬取信息： 3055
main: None
已爬取信息： 已爬取信息：3055 
已爬取信息： 3055
main: None3055
main: None

main: None
已爬取信息： 3055
main: None
已爬取信息： 已爬取信息：3055
main: None 3055

main: None
已爬取信息： 3055
main: None
已爬取信息： 3055
main: None
list index out of range
main: None
已爬取信息： 3065
main: None已爬取信息：
 3065
main: None
已爬取信息： 3065
main: None
已爬取信息：已爬取信息：  3065
已爬取信息： 3065
main: None
main: None
3065
main: None
已爬取信息： 3065
main: None
已爬取信息： 3065
main: None
已爬取信息： 3065
main: None
已爬取信息： 3065
main: None
list index out of range
main: None
已爬取信息： 3075
main: None
已爬取信息： 3075
main: None已爬取信息：
 3075
main: None
已爬取信息： 3075
main: None
已爬取信息： 3075
main: None
已爬取信息： 3

Was a nice sleep, now let me continue...
main: None
当前正在爬取: 第259页
当前正在爬取: https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,259.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
main: None
list index out of range
list index out of rangemain: None

main: None
main: None
已爬取信息： 3255
main: None
已爬取信息： 3256
main: None
已爬取信息： 3256
main: None
已爬取信息： 3256
main: None
已爬取信息： 3256
main: None
已爬取信息： 3256
main: None
已爬取信息：已爬取信息： 3256 
3256main: None

main: None
list index out of range
main: None
已爬取信息： 3256
main: None
已爬取信息： 3259
list index out of range
main: None
main: None
已爬取信息：已爬取信息：  3260已爬取信息： 
3260main: None3260


main: None
main: None
已爬取信息： 3260
main: None
list index out of

HTTPSConnectionPool(host='jobs.51job.com', port=443): Max retries exceeded with url: /all/co404548.html (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002A766FC5648>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))
Connection refused by the server..
Let me sleep for 5 seconds
Was a nice sleep, now let me continue...
main: None
Was a nice sleep, now let me continue...
main: None
Was a nice sleep, now let me continue...Was a nice sleep, now let me continue...

main: None
main: None
已爬取信息： 3335
main: None
已爬取信息： 3335
main: None
已爬取信息：已爬取信息： 3335
 3335main: None

main: None
Was a nice sleep, now let me continue...
main: None
Was a nice sleep, now let me continue...
main: None
main: None
Was a nice sleep, now let me continue...
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
已爬取信息： 3341
main: None
list index out of range
main:

Was a nice sleep, now let me continue...
main: None
当前正在爬取: 第266页
当前正在爬取: https://search.51job.com/list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,266.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare=
爬取公司链接时 'NoneType' object is not subscriptable
list index out of range
list index out of rangemain: None

main: Nonelist index out of range

main: None
已爬取信息： 3526
main: None
已爬取信息：已爬取信息：  35263526
main: None

main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
已爬取信息： 3526
main: None
list index out of range
main: None
main: None
已爬取信息： 3535
main: None
已爬取信息：已爬取信息：已爬取信息： 3535
 main: None3535
 
3535
main: None
main: None
已爬取信息： 3535已爬取信息： 3535

main: None
main: None
已爬取信息：

ConnectionError: HTTPSConnectionPool(host='search.51job.com', port=443): Max retries exceeded with url: /list/000000,000000,0000,00,9,99,%25E6%2595%25B0%25E6%258D%25AE%25E5%2588%2586%25E6%259E%2590,2,270.html?lang=c&stype=&postchannel=0000&workyear=99&cotype=99&degreefrom=99&jobterm=99&companysize=99&providesalary=99&lonlat=0%2C0&radius=-1&ord_field=0&confirmdate=9&fromType=&dibiaoid=0&address=&line=&specialarea=00&from=&welfare= (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000002A765697EC8>: Failed to establish a new connection: [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。'))